In [14]:
from IPython.utils import io
import csv
with io.capture_output() as captured:  
   %run ../../2-FeatureExtraction/TF-IDF/TF-IDF.ipynb import x, y1, y2, X, Y1, Y2, vocabulary, Preprocessing, Features

In [15]:
from sklearn import svm
from sklearn.metrics import f1_score, classification_report

#### SVM Training

In [16]:
kernel = 'poly'
WeightedLoss = False
model = svm.SVC(kernel=kernel, class_weight='balanced' if WeightedLoss else None )
model = model.fit(x, y1)
ModelInfo = {
   "Model": "SVM-1",
   "Kernel":kernel,
   "Weighted Loss": WeightedLoss
}

#### SVM Prediction

In [17]:
outputs = model.predict(X)
Report = classification_report(Y1, outputs, output_dict=True)
print(classification_report(Y1, outputs))
F1 = f1_score(Y1, outputs, average='macro')
print('F1 score: ', F1)


              precision    recall  f1-score   support

           0       0.34      0.52      0.41       126
           1       0.90      0.85      0.87       804
           2       0.44      0.33      0.38        70

    accuracy                           0.77      1000
   macro avg       0.56      0.57      0.55      1000
weighted avg       0.80      0.77      0.78      1000

F1 score:  0.5543410952715655


#### Log Model Info

In [18]:
with open('runs.csv', 'a') as f:  
      run_info = {**Preprocessing, **Features, **ModelInfo, "acc":Report["accuracy"],"BF1": Report["macro avg"]["f1-score"], "WF1": Report["weighted avg"]["f1-score"] }
      w = csv.DictWriter(f, run_info.keys())
      w.writeheader()
      w.writerow(run_info)